# Cleanup and Merges

This notebook takes the data, cleans it, compiles it into the datasets we need for our analysis, and saves them in this project's datafolder

There are 4 datasets:
- Orders (9994 rows)
- Order Process Data (5899 rows)
- Campaign Data (333 rows)
- InternData Study (204 rows)

And four events for which we have data:
1. Order date
2. Ready for shipping
3. On Truck Scan
4. Arrival Scan

This gives us 3 consecutive timespans, as well as the general leadtime.

We will perform the following merges to cover these timespans:

- Leadtime (order date --> arrival scan): Orders merged with Campaign Data

- Warehouse (order date --> ready to ship): Orders merged with Intern Data

- Pickuptime (ready to ship --> Pickup/on truck): The InternData set has both ready to ship and pick up dates

- Delivery (pickup/on truck --> arrival scan): Order Process merged with Campaign Data


As the inital EDA revealed surprising behaviour of orders for which the shipmode is "Express", we add the shipmode-column to each dataframe we're working with.


## Clean up

The cleanup is done with one function (see below). We are only missing 11 postal codes from df_orders. This doesn't interfere with the following analysis, so we ignore them.
The function performs the following:

1. Lowercasing column names
2. Replacing spaces in column names with underscores
3. Checking and printing the number of duplicate rows
4. Dropping duplicate rows
5. Printing the number of missing values per column
6. Returning a summary of the DataFrame after cleanup

## Merges

All datasets feature a column of order_id, but the dataframe's rows correspond to items, not orders. If an order comprises more than one type of item, there will be multiple rows per order_id. As we only care for the timeline, not the order's contents, for now we proceed with keeping only one row per order_id.

However, as the datasets vary in size, the only robust timespan is between *order date* and *on-truck scan*. Both *ready for shipping* and *arrival scan* are part of datasets with ~200 and ~300 rows, respectively, and any combination with other events, such as order date to arrival scan will be bottlenecked by the smaller dataset.



In [2]:
## Import 

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
## EXTRACT:

df_orders = pd.read_excel("data/muesli_project_raw_data.xlsx", header=1, sheet_name="Orders")
df_order_process_data = pd.read_excel("data/muesli_project_raw_data.xlsx", header=0, sheet_name="Order Process Data")
df_campaign_data = pd.read_excel("data/muesli_project_raw_data.xlsx", header=0, sheet_name="Campaign Data")
df_intern_data_study = pd.read_excel("data/muesli_project_raw_data.xlsx", header=0, sheet_name="InternData Study")

In [4]:
## Taking a first look

df_orders.head()

,Index,Order ID,Order Date,Ship Mode,Customer ID,Customer Name,Origin Channel,Country/Region,City,State,Postal Code,Region,Category,Sub-Category,Product ID,Sales,Quantity,Discount,Profit
0,27,CA-2019-121755,2019-01-16,Second Class,EH-13945,Eric Hoffmann,Email,United States,Los Angeles,California,90049.0,West,Special Projects Muesil,Gluten Free,TEC-AC-10003027,90.57,3,0.0,11.7741
1,45,CA-2019-118255,2019-03-11,First Class,ON-18715,Odella Nelson,Sales,United States,Eagan,Minnesota,55122.0,Central,Special Projects Muesil,Gluten Free,TEC-AC-10000171,45.98,2,0.0,19.7714
2,48,CA-2019-169194,2019-06-20,Standard Class,LH-16900,Lena Hernandez,Email,United States,Dover,Delaware,19901.0,East,Special Projects Muesil,Gluten Free,TEC-AC-10002167,45.00,3,0.0,4.9500
3,60,CA-2019-111682,2019-06-17,First Class,TB-21055,Ted Butterfield,Email,United States,Troy,New York,12180.0,East,Special Projects Muesil,Gluten Free,TEC-AC-10002167,30.00,2,0.0,3.3000
4,63,CA-2018-135545,2018-11-24,Standard Class,KM-16720,Kunst Miller,Email,United States,Los Angeles,California,90004.0,West,Special Projects Muesil,Gluten Free,TEC-AC-10004633,13.98,2,0.0,6.1512


In [5]:
### TRANSFORM

## Function to clean up each dataframe

def cleanup(data: pd.DataFrame) -> pd.DataFrame:
    """
    Clean up the input DataFrame by:
    1. Lowercasing column names
    2. Replacing spaces in column names with underscores
    3. Checking and printing the number of duplicate rows
    4. Dropping duplicate rows
    5. Printing the number of missing values per column
    6. Returning a summary of the DataFrame after cleanup
    
    Parameters:
    - data (pd.DataFrame): The DataFrame to clean up.
    
    Returns:
    - pd.DataFrame: The cleaned DataFrame.
    """
    
    # Step 1: Clean column names: lowercase and underscore instead of spaces
    data.columns = (data.columns.str.lower()
                    .str.replace(" ", "_"))
    
    # Step 2: Handle duplicates
    duplicate_rows = data[data.duplicated()]
    print(f"There are {len(duplicate_rows)} duplicate rows.")
    
    # Drop duplicates
    data.drop_duplicates(inplace=True, ignore_index=True)
    
    # Step 3: Check for NaN values
    missing_values = data.isna().sum()
    print(f"Missing values per column:\n{missing_values}")
    
    # Step 4: Return cleaned data and additional info
    print(f"Summary after cleanup:\n")
    data_info = data.info()  
    
    
    return data  

In [6]:
cleanup(df_orders)

There are 0 duplicate rows.
Missing values per column:
index              0
order_id           0
order_date         0
ship_mode          0
customer_id        0
customer_name      0
origin_channel     0
country/region     0
city               0
state              0
postal_code       11
region             0
category           0
sub-category       0
product_id         0
sales              0
quantity           0
discount           0
profit             0
dtype: int64
Summary after cleanup:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9994 entries, 0 to 9993
Data columns (total 19 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   index           9994 non-null   int64         
 1   order_id        9994 non-null   object        
 2   order_date      9994 non-null   datetime64[ns]
 3   ship_mode       9994 non-null   object        
 4   customer_id     9994 non-null   object        
 5   customer_name   9994 non-null   o

,index,order_id,order_date,ship_mode,customer_id,customer_name,origin_channel,country/region,city,state,postal_code,region,category,sub-category,product_id,sales,quantity,discount,profit
0,27,CA-2019-121755,2019-01-16,Second Class,EH-13945,Eric Hoffmann,Email,United States,Los Angeles,California,90049.0,West,Special Projects Muesil,Gluten Free,TEC-AC-10003027,90.570,3,0.0,11.7741
1,45,CA-2019-118255,2019-03-11,First Class,ON-18715,Odella Nelson,Sales,United States,Eagan,Minnesota,55122.0,Central,Special Projects Muesil,Gluten Free,TEC-AC-10000171,45.980,2,0.0,19.7714
2,48,CA-2019-169194,2019-06-20,Standard Class,LH-16900,Lena Hernandez,Email,United States,Dover,Delaware,19901.0,East,Special Projects Muesil,Gluten Free,TEC-AC-10002167,45.000,3,0.0,4.9500
3,60,CA-2019-111682,2019-06-17,First Class,TB-21055,Ted Butterfield,Email,United States,Troy,New York,12180.0,East,Special Projects Muesil,Gluten Free,TEC-AC-10002167,30.000,2,0.0,3.3000
4,63,CA-2018-135545,2018-11-24,Standard Class,KM-16720,Kunst Miller,Email,United States,Los Angeles,California,90004.0,West,Special Projects Muesil,Gluten Free,TEC-AC-10004633,13.980,2,0.0,6.1512
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9989,9894,US-2019-115441,2019-07-25,Second Class,SH-19975,Sally Hughsby,Sales,United States,Milwaukee,Wisconsin,53209.0,Central,Toasted Muesli,With Nuts,FUR-CH-10004626,403.560,4,0.0,96.8544
9990,9908,US-2018-129007,2018-09-13,First Class,KD-16615,Ken Dana,Sales,United States,Anaheim,California,92804.0,West,Toasted Muesli,With Nuts,FUR-CH-10000155,717.720,3,0.2,71.7720
9991,9913,CA-2018-132388,2018-10-10,First Class,KN-16390,Katherine Nockton,Sales,United States,Santa Barbara,California,93101.0,West,Toasted Muesli,With Nuts,FUR-CH-10001714,362.136,3,0.2,-54.3204
9992,9920,CA-2019-149272,2019-03-15,Standard Class,MY-18295,Muhammed Yedwab,Sales,United States,Bryan,Texas,77803.0,Central,Toasted Muesli,With Nuts,FUR-CH-10000863,528.430,5,0.3,-143.4310


In [7]:
cleanup(df_order_process_data)

There are 0 duplicate rows.
Missing values per column:
row_id                0
order_id              0
order_date            0
on_truck_scan_date    0
ship_mode             0
dtype: int64
Summary after cleanup:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5899 entries, 0 to 5898
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   row_id              5899 non-null   int64         
 1   order_id            5899 non-null   object        
 2   order_date          5899 non-null   datetime64[ns]
 3   on_truck_scan_date  5899 non-null   datetime64[ns]
 4   ship_mode           5899 non-null   object        
dtypes: datetime64[ns](2), int64(1), object(2)
memory usage: 230.6+ KB


,row_id,order_id,order_date,on_truck_scan_date,ship_mode
0,3074,CA-2019-125206,2019-01-03,2019-01-07,Express
1,4919,CA-2019-160304,2019-01-02,2019-01-09,Standard Processing
2,4920,CA-2019-160304,2019-01-02,2019-01-09,Standard Processing
3,8604,US-2019-116365,2019-01-03,2019-01-09,Standard Processing
4,8605,US-2019-116365,2019-01-03,2019-01-09,Standard Processing
...,...,...,...,...,...
5894,908,CA-2020-143259,2020-12-30,2021-01-06,Standard Processing
5895,909,CA-2020-143259,2020-12-30,2021-01-06,Standard Processing
5896,1297,CA-2020-115427,2020-12-30,2021-01-06,Standard Processing
5897,1298,CA-2020-115427,2020-12-30,2021-01-06,Standard Processing


In [8]:
cleanup(df_campaign_data)

There are 0 duplicate rows.
Missing values per column:
order_id             0
arrival_scan_date    0
customer_name        0
dtype: int64
Summary after cleanup:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 333 entries, 0 to 332
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   order_id           333 non-null    object        
 1   arrival_scan_date  333 non-null    datetime64[ns]
 2   customer_name      333 non-null    object        
dtypes: datetime64[ns](1), object(2)
memory usage: 7.9+ KB


,order_id,arrival_scan_date,customer_name
0,CA-2019-109666,2019-05-03,Kunst Miller
1,CA-2019-138933,2019-05-03,Jack Lebron
2,CA-2019-130001,2019-05-03,Heather Kirkland
3,CA-2019-113061,2019-05-06,Ed Ludwig
4,CA-2019-162138,2019-05-06,Grace Kelly
...,...,...,...
328,CA-2020-129707,2020-05-08,Larry Hughes
329,CA-2020-125381,2020-05-08,Speros Goranitis
330,CA-2020-141733,2020-05-15,Rick Wilson
331,US-2020-104451,2020-05-15,Michelle Moray


In [9]:
cleanup(df_intern_data_study)

There are 86 duplicate rows.
Missing values per column:
order_id              0
ready_to_ship_date    0
pickup_date           0
dtype: int64
Summary after cleanup:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 204 entries, 0 to 203
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   order_id            204 non-null    object        
 1   ready_to_ship_date  204 non-null    datetime64[ns]
 2   pickup_date         204 non-null    datetime64[ns]
dtypes: datetime64[ns](2), object(1)
memory usage: 4.9+ KB


,order_id,ready_to_ship_date,pickup_date
0,CA-2019-116540,2019-09-02,2019-09-03
1,CA-2019-129847,2019-09-04,2019-09-04
2,CA-2019-129630,2019-09-04,2019-09-04
3,CA-2019-106278,2019-09-05,2019-09-06
4,CA-2019-158099,2019-09-05,2019-09-06
...,...,...,...
199,US-2020-165456,2020-12-03,2020-12-04
200,US-2020-110576,2020-12-04,2020-12-07
201,CA-2020-105333,2020-12-04,2020-12-07
202,CA-2020-119305,2020-12-04,2020-12-07


## Preparing the Merges: Unique order_id's

In order to merge on each dataset's order_id, we want to make sure that each dataset has unique order_id's 

In [10]:
print("orders needs to be fixed") if df_orders["order_id"].nunique() != len(df_orders) else print("orders are good")

print("order_process needs to be fixed") if df_order_process_data["order_id"].nunique() != len(df_order_process_data) else print("order_process is good")

print("intern needs to be fixed") if df_intern_data_study["order_id"].nunique() != len(df_intern_data_study) else print("intern is good")

print("campaign needs to be fixed") if df_campaign_data["order_id"].nunique() != len(df_campaign_data) else print("campaign is good")


orders needs to be fixed
order_process needs to be fixed
intern is good
campaign is good



For df_orders: It turns out that order_id's are not unique, as each row corresponds to an item bought. If an order comprises more than one item, that order_id will appear for every type of item bought within that order. 

For our purposes, as we don't care for the contents of the order (for now at least), we remove duplicates of order id's. Then we merge.

But first, let's doublecheck whether there is only ever one date per oder_id:

In [11]:
# First, we group by 'order_id' and check the number of unique dates per order
duplicate_dates = df_orders.groupby("order_id")["order_date"].nunique()

# Our hypothesis is that there are no order_id's with more than one unique date. 
# We filter to find order_ids that have more than one unique date
multiple_dates = duplicate_dates[duplicate_dates > 1]

# Show the order_ids with multiple dates
multiple_dates

##Ergo, we can proceed with keeping only one row per order_id

Series([], Name: order_date, dtype: int64)

In [12]:
## Aggregation: Turn multiple rows per order_id into one row per order_id


##we aggregate the columns with "sum"
#except for the orderdate, where we display the latest (which is safe, since we have checked that there is always only one date per order_id)

#in summing the amounts, we still keep the information how many items an order comprised. However, we lose the information how many types of items were bought.

agg_funcs = {
    "order_date": "max",  # Get the "latest" order date
    "sales": "sum",       # Sum the amounts
    "quantity": "sum",       # Sum the amounts
    "discount": "sum",       # Sum the amounts
    "profit": "sum"       # Sum the amounts

}

#df_orders_grouped is ready to be merged
df_orders_grouped = df_orders.groupby("order_id").agg(agg_funcs).reset_index()

In [13]:
## Taking a look
df_orders_grouped.head()

## and save the clean dataset for later use:
df_orders_grouped.to_csv("data/orders.csv", index=False)


For df_order_process_data: row_id makes otherwise duplicate rows to be different, without containing relevant information. We drop it and remove duplicates by running our cleanup function again

In [14]:
#dropping row_id, which causes otherwise duplicate rows to evade our cleanup-function
df_order_process_data.drop(columns=["row_id"], inplace=True)

In [15]:
#run the cleanup again
cleanup(df_order_process_data)

There are 2896 duplicate rows.
Missing values per column:
order_id              0
order_date            0
on_truck_scan_date    0
ship_mode             0
dtype: int64
Summary after cleanup:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3003 entries, 0 to 3002
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   order_id            3003 non-null   object        
 1   order_date          3003 non-null   datetime64[ns]
 2   on_truck_scan_date  3003 non-null   datetime64[ns]
 3   ship_mode           3003 non-null   object        
dtypes: datetime64[ns](2), object(2)
memory usage: 94.0+ KB


,order_id,order_date,on_truck_scan_date,ship_mode
0,CA-2019-125206,2019-01-03,2019-01-07,Express
1,CA-2019-160304,2019-01-02,2019-01-09,Standard Processing
2,US-2019-116365,2019-01-03,2019-01-09,Standard Processing
3,CA-2019-105207,2019-01-03,2019-01-09,Standard Processing
4,CA-2019-158211,2019-01-04,2019-01-09,Standard Processing
...,...,...,...,...
2998,CA-2020-130631,2020-12-29,2021-01-06,Standard Processing
2999,CA-2020-126221,2020-12-30,2021-01-06,Standard Processing
3000,CA-2020-143259,2020-12-30,2021-01-06,Standard Processing
3001,CA-2020-115427,2020-12-30,2021-01-06,Standard Processing


Let's check again if we fixed everything:

In [16]:
print("orders needs to be fixed") if df_orders_grouped["order_id"].nunique() != len(df_orders_grouped) else print("orders are good")

print("order_process needs to be fixed") if df_order_process_data["order_id"].nunique() != len(df_order_process_data) else print("order_process is good")

print("intern needs to be fixed") if df_intern_data_study["order_id"].nunique() != len(df_intern_data_study) else print("intern is good")

print("campaign needs to be fixed") if df_campaign_data["order_id"].nunique() != len(df_campaign_data) else print("campaign is good")

orders are good
order_process needs to be fixed
intern is good
campaign is good


In [17]:
#since there are still duplicate order_id's in order_process, let's take a closer look

duplicate_orders = df_order_process_data[df_order_process_data.duplicated(subset=["order_id"], keep=False)]
duplicate_orders

,order_id,order_date,on_truck_scan_date,ship_mode
2140,CA-2020-101182,2020-09-04,2020-09-07,Express
2141,CA-2020-101182,2020-09-04,2020-09-08,Express


In [18]:
# The same order_id has been scanned twice, on two consecutive days. 
# We'll keep the earlier one, as it had to have been on a truck to be scanned on a truck, and judge the other row to be erroneous.

df_order_process_data.drop(2141, inplace=True)

print("order_process needs to be fixed") if df_order_process_data["order_id"].nunique() != len(df_order_process_data) else print("order_process is good")


order_process is good


### Merging: Lead Time (Order --> Arrival Scan)


In [19]:
## Merging: Orders and Campaign --> This gives us the order date and arrival scan.
#(bottlenecked by the smaller campaign set to 333 rows only)

df_lead = df_orders_grouped.merge(df_campaign_data, how="inner", on="order_id")
df_lead.head()

,order_id,order_date,sales,quantity,discount,profit,arrival_scan_date,customer_name
0,CA-2019-100041,2019-11-20,328.540,6,0.0,157.3777,2019-12-02,Barbara Fisher
1,CA-2019-100083,2019-11-24,24.784,1,0.2,7.7450,2019-12-09,Carol Darley
2,CA-2019-100244,2019-09-20,475.694,19,0.4,175.6262,2019-09-30,Greg Maxwell
3,CA-2019-100468,2019-11-24,43.460,4,0.2,6.4334,2019-12-09,Alyssa Tate
4,CA-2019-100510,2019-05-12,641.980,7,0.0,307.7496,2019-05-28,Harry Marie


In [20]:
#add shipmode to df_lead:
df_lead = df_lead.merge(df_order_process_data[["order_id", "ship_mode"]], on="order_id", how="left")

In [21]:
## LOAD: locally as csv:

df_lead.to_csv("data/leadtime.csv", index=False)

### Merging: Warehouse Time (Order --> Ready to Ship)

- The InternDataSet has the data for when orders are ready to ship and when they are picked up
- OrderProcesses has a row_id column that causes otherwise duplicate rows not to be flagged as duplicates. We remove the row and run our cleanup-function again

In [22]:
df_intern_data_study.head()

,order_id,ready_to_ship_date,pickup_date
0,CA-2019-116540,2019-09-02,2019-09-03
1,CA-2019-129847,2019-09-04,2019-09-04
2,CA-2019-129630,2019-09-04,2019-09-04
3,CA-2019-106278,2019-09-05,2019-09-06
4,CA-2019-158099,2019-09-05,2019-09-06


In [23]:
df_warehouse = df_intern_data_study.merge(df_order_process_data, how="inner", on="order_id")

#as we merge with order_process anyway, no need to add the shipmode-column. Its already there.

In [24]:
df_warehouse.to_csv("data/warehouse.csv", index=False)

### Merging: Pick up time

No need, since both *ready to ship* and *pick up* are contained in the Intern Data set. Still, we add the "ship_mode" column to verify the assumption that express-shippings are shipped the day they are ready to ship.





In [25]:
#add shipmode to df_intern:
df_intern_data_study = df_intern_data_study.merge(df_order_process_data[["order_id", "ship_mode"]], on="order_id", how="left")

In [26]:
df_intern_data_study.to_csv("data/pickup.csv", index=False)

### Merging: Delivery

We'll use the OrderProcess data for *on truck scans* and the CampaignData set for the *arrival scan*

In [27]:
df_delivery = df_order_process_data.merge(df_campaign_data, how="inner", on="order_id")

In [28]:
#add shipmode to df_delivery:
df_delivery = df_delivery.merge(df_order_process_data[["order_id", "ship_mode"]], on="order_id", how="left")

In [29]:
df_delivery.to_csv("data/delivery.csv", index=False)

### The End

This concludes cleaning and merging the datasets in order to perform an inital EDA in the next notebook.

